<a href="https://colab.research.google.com/github/Nataliia-Kaguia/goit-ds-hw-10/blob/main/hw_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y scikit-surprise
!pip install numpy==1.26.4 cython
!pip install git+https://github.com/NicolasHug/Surprise.git


  Cloning https://github.com/NicolasHug/Surprise.git to /tmp/pip-req-build-psuymg1o
  Running command git clone --filter=blob:none --quiet https://github.com/NicolasHug/Surprise.git /tmp/pip-req-build-psuymg1o
  Resolved https://github.com/NicolasHug/Surprise.git to commit 2381fb11d0c4bf917cc4b9126f205d0013649966
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2610171 sha256=5fa8c34a74ee4057de42fcf7bb7ac0c039cbfe2a4883db6d066b15a624d360b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-sju9z7v3/wheels/81/ef/f8/c6e173e03432de2eeaa047537fada2fa9b7cf37559598634f0
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV

# ===============================
# 1. Завантажуємо дані MovieLens 100k
# ===============================
data = Dataset.load_builtin("ml-100k")


Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
# ===============================
# 2. GridSearchCV для підбору параметрів SVD
# ===============================
param_grid = {
    "n_epochs": [5, 10, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.2, 0.4]
}

gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3, n_jobs=-1)
gs.fit(data)

print("Найкращі параметри для SVD:", gs.best_params["rmse"])
print("Найкраща оцінка RMSE:", gs.best_score["rmse"])


Найкращі параметри для SVD: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}
Найкраща оцінка RMSE: 0.9493295804791196


In [ ]:
# ===============================
# 3. Порівняння алгоритмів
# ===============================
algorithms = {
    "SVD (оптимізований)": SVD(**gs.best_params["rmse"]),
    "SVD++": SVDpp(),
    "NMF": NMF()
}

results = {}
for name, algo in algorithms.items():
    cv_results = cross_validate(algo, data, measures=["rmse", "mae"], cv=3, verbose=False)
    results[name] = {
        "RMSE": cv_results["test_rmse"].mean(),
        "MAE": cv_results["test_mae"].mean()
    }

# Вивід результатів
print("\nПорівняння алгоритмів:")
for name, metrics in results.items():
    print(f"{name}: RMSE={metrics['RMSE']:.4f}, MAE={metrics['MAE']:.4f}")



Порівняння алгоритмів:
SVD (оптимізований): RMSE=0.9492, MAE=0.7558
SVD++: RMSE=0.9269, MAE=0.7282
NMF: RMSE=0.9785, MAE=0.7698


Завдання з зірочкою

In [ ]:
import numpy as np
import scipy.io
import requests

# === Завантажуємо .mat файл з GitHub (матриця рейтингів) ===
url_mat = "https://raw.githubusercontent.com/Nataliia-Kaguia/goit-ds-hw-10/main/movies.mat"
r = requests.get(url_mat)
with open("movies.mat", "wb") as f:
    f.write(r.content)

data = scipy.io.loadmat("movies.mat")
Y = data["Y"]   # матриця рейтингів (num_movies x num_users)
R = data["R"]   # індикатор: 1 якщо є рейтинг, 0 якщо немає

print("Форма Y (ratings):", Y.shape)
print("Форма R (indicator):", R.shape)

# === Завантажуємо movie_ids.txt ===
url_txt = "https://raw.githubusercontent.com/Nataliia-Kaguia/goit-ds-hw-10/main/movie_ids.txt"
r = requests.get(url_txt)
movie_list = r.text.split("\n")

print("Кількість фільмів:", len(movie_list))
print("Перші 10 назв:", movie_list[:10])


Форма Y (ratings): (1682, 943)
Форма R (indicator): (1682, 943)
Кількість фільмів: 1683
Перші 10 назв: ['1 Toy Story (1995)', '2 GoldenEye (1995)', '3 Four Rooms (1995)', '4 Get Shorty (1995)', '5 Copycat (1995)', '6 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', '7 Twelve Monkeys (1995)', '8 Babe (1995)', '9 Dead Man Walking (1995)', '10 Richard III (1995)']


In [ ]:
import numpy as np
import scipy.io
import requests

# === Крок 1. Завантаження даних ===
# Завантажуємо .mat (рейтинги)
url_mat = "https://raw.githubusercontent.com/Nataliia-Kaguia/goit-ds-hw-10/main/movies.mat"
r = requests.get(url_mat)
with open("movies.mat", "wb") as f:
    f.write(r.content)

data = scipy.io.loadmat("movies.mat")
Y = data["Y"]   # матриця рейтингів (num_movies x num_users)
R = data["R"]   # індикатор: 1 якщо є рейтинг, 0 якщо немає

num_movies, num_users = Y.shape
print("Форма Y:", Y.shape, "Форма R:", R.shape)


Форма Y: (1682, 943) Форма R: (1682, 943)


In [ ]:
# Завантажуємо список фільмів
url_txt = "https://raw.githubusercontent.com/Nataliia-Kaguia/goit-ds-hw-10/main/movie_ids.txt"
r = requests.get(url_txt)
movie_list = r.text.split("\n")
print("Кількість фільмів:", len(movie_list))
print("Перші 5:", movie_list[:5])


Кількість фільмів: 1683
Перші 5: ['1 Toy Story (1995)', '2 GoldenEye (1995)', '3 Four Rooms (1995)', '4 Get Shorty (1995)', '5 Copycat (1995)']


In [ ]:
# === Крок 2. Матрична факторизація ===
def matrix_factorization(Y, R, k=10, alpha=0.001, reg=0.02, epochs=50):
    """
    Y - матриця рейтингів (num_movies x num_users)
    R - індикатор (1 якщо є рейтинг, 0 якщо немає)
    k - кількість латентних факторів
    alpha - learning rate
    reg - коефіцієнт регуляризації
    epochs - кількість епох
    """

    num_movies, num_users = Y.shape

    # Ініціалізація матриць
    U = np.random.normal(scale=1./k, size=(num_users, k))
    V = np.random.normal(scale=1./k, size=(num_movies, k))

    # Історія втрат
    history = []

    for epoch in range(epochs):
        # Обновлюємо U і V
        for i in range(num_movies):
            for j in range(num_users):
                if R[i, j] == 1:  # тільки де є рейтинг
                    eij = Y[i, j] - np.dot(U[j, :], V[i, :].T)

                    # Градієнтний спуск
                    U[j, :] += alpha * (eij * V[i, :] - reg * U[j, :])
                    V[i, :] += alpha * (eij * U[j, :] - reg * V[i, :])

        # Обчислюємо помилку після кожної епохи
        error = 0
        for i in range(num_movies):
            for j in range(num_users):
                if R[i, j] == 1:
                    error += (Y[i, j] - np.dot(U[j, :], V[i, :].T)) ** 2
                    error += reg * (np.linalg.norm(U[j, :]) + np.linalg.norm(V[i, :]))
        history.append(error)

        if epoch % 10 == 0:
            print(f"Епоха {epoch}/{epochs}, помилка: {error:.4f}")

    return U, V, history


In [ ]:
# === Крок 3. Навчання ===
U, V, history = matrix_factorization(Y, R, k=10, alpha=0.002, reg=0.02, epochs=50)

# === Крок 4. Робимо передбачення ===
pred_matrix = np.dot(U, V.T)  # num_users x num_movies

user_id = 0  # подивимось рекомендації для користувача №1
top_movies = np.argsort(-pred_matrix[user_id, :])[:10]

print("\nРекомендовані фільми для користувача 1:")
for idx in top_movies:
    print(movie_list[idx])


Епоха 0/50, помилка: 1371517.5475
Епоха 10/50, помилка: 117473.9880
Епоха 20/50, помилка: 96322.4760
Епоха 30/50, помилка: 90384.6726
Епоха 40/50, помилка: 85606.8080

Рекомендовані фільми для користувача 1:
408 Close Shave, A (1995)
56 Pulp Fiction (1994)
12 Usual Suspects, The (1995)
127 Godfather, The (1972)
169 Wrong Trousers, The (1993)
511 Lawrence of Arabia (1962)
357 One Flew Over the Cuckoo's Nest (1975)
474 Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
513 Third Man, The (1949)
134 Citizen Kane (1941)
